In [18]:
import glob
import pandas as pd
import numpy as np
import statistics as st
import matplotlib.pyplot as plt
import re
from DataProcessing import *
from DataImporting import sequence_list
from venditti_data_importer import venditti_assignment
peak_list, residue_list = main_data_processing()

acceptance_threshold = .20
should_print_unassigned_lists = False
should_print_peak_data = True
should_export_to_excel = True
should_take_best_10 = False


# if you want to analyze multiple test conditions, add them here.
for random_index in range(1):
    # DATA INGESTION
    time_taken_list = list()
    final_energy_list = list()
    filename_list = list()
    count_of_assignments = 0

    # glob.glob returns a list of paths that match filename NAME.
    # add all files for each test condition

    # NAME = ["Slurm Trials/slurm-4102660_{}.out".format(str(x)) for x in range(1, 17)] + \
    #        ["Slurm Trials/slurm-4109433_{}.out".format(str(x)) for x in range(17, 201)] + \
    #        ["Slurm Trials/slurm-9199622_{}.out".format(str(x)) for x in range(1, 81)]

    NAME = ["40:30.out", "40:30_2.out"]



    set_iterations, initial_temperature, exponent, eif = None, None, None, None
    time_taken, completed_iterations, final_temp, og_index_list, og_energy, index_list, final_energy = \
        None, None, None, None, None, None, None

    initial_data = pd.DataFrame(columns=['set_iterations', 'time_taken', 'completed_iterations', 'final_temp',
                                         'og_index_list', 'og_energy', 'index_list', 'final_energy'])

    for filename_str in NAME:
        # imports each file
        # filename = [i[0] for i in glob.glob(filename_str)]
        file = open(filename_str, 'r')
        filename_list.append(filename_str)

        for line in file.readlines():
            if "Done" in line:
                count_of_assignments += 1
                df_temp = pd.DataFrame([[set_iterations, time_taken, completed_iterations,
                                         final_temp, og_index_list, og_energy,
                                         index_list, final_energy]],
                                       columns=['set_iterations', 'time_taken', 'completed_iterations',
                                                'final_temp', 'og_index_list', 'og_energy',
                                                'index_list', 'final_energy'])
                initial_data = initial_data.append(df_temp, ignore_index=True)

                set_iterations, initial_temperature, exponent, eif = None, None, None, None
                time_taken, completed_iterations, final_temp, og_index_list, og_energy, index_list, final_energy = \
                    None, None, None, None, None, None, None

            elif 'exponent:' in line:
                set_iterations = int(re.search(r" \d* ", line)[0][1:-1])
            elif 'time taken (sec)' in line:
                time_taken = float(re.search(r"\d*\.\d*", line)[0])
                time_taken_list.append(time_taken)
            elif 'number of iterations' in line:
                completed_iterations = int(re.search(r"[0-9].*", line)[0])
            elif 'final temperature' in line:
                final_temp = float(re.search(r"\d*\.\d*", line)[0])
                final_energy_list.append(final_temp)
            elif 'og index list:' in line:
                og_index_list = list()
                [og_index_list.append(int(x)) for x in line[line.index('[') + 1:line.index(']')].split(', ')]
            elif 'og index list energy:' in line:
                og_energy = float(re.search(r"\d*\.\d*", line)[0])
            elif 'index list:' in line:
                index_list = list()
                [index_list.append(int(x)) for x in line[line.index('[') + 1:line.index(']')].split(', ')]
            elif 'index list energy:' in line:
                final_energy = float(re.search(r"\d*\.\d*", line)[0])

    print(initial_data.head())
        # print(time_taken, completed_iterations, final_temp, og_index_list, og_energy, index_list, final_energy)

  set_iterations    time_taken completed_iterations     final_temp  \
0      900000000  55733.223443            900000000     159.267911   
1      900000000  57654.434814            900000000     159.267911   
2      900000000  56867.216628            900000000     159.267911   
3      900000000   1944.742764             29947115  656100.000000   
4      900000000  62962.210565            900000000     159.267911   

                                       og_index_list     og_energy  \
0  [109, 111, 463, 200, 56, 144, 456, 289, 307, 2...  8.630539e+07   
1  [340, 411, 384, 90, 288, 44, 425, 2, 334, 178,...  1.202441e+08   
2  [73, 405, 145, 63, 463, 13, 300, 111, 377, 276...  1.080344e+08   
3  [254, 247, 136, 233, 332, 388, 127, 164, 380, ...  1.059793e+08   
4  [9, 182, 469, 217, 173, 428, 265, 353, 113, 33...  1.501029e+08   

                                          index_list  final_energy  
0  [116, 282, 452, 72, 67, 436, 447, 199, 206, 41...  1.647113e+06  
1  [116, 80, 257, 21

In [19]:
print(initial_data.head())

  set_iterations    time_taken completed_iterations     final_temp  \
0      900000000  55733.223443            900000000     159.267911   
1      900000000  57654.434814            900000000     159.267911   
2      900000000  56867.216628            900000000     159.267911   
3      900000000   1944.742764             29947115  656100.000000   
4      900000000  62962.210565            900000000     159.267911   

                                       og_index_list     og_energy  \
0  [109, 111, 463, 200, 56, 144, 456, 289, 307, 2...  8.630539e+07   
1  [340, 411, 384, 90, 288, 44, 425, 2, 334, 178,...  1.202441e+08   
2  [73, 405, 145, 63, 463, 13, 300, 111, 377, 276...  1.080344e+08   
3  [254, 247, 136, 233, 332, 388, 127, 164, 380, ...  1.059793e+08   
4  [9, 182, 469, 217, 173, 428, 265, 353, 113, 33...  1.501029e+08   

                                          index_list  final_energy  
0  [116, 282, 452, 72, 67, 436, 447, 199, 206, 41...  1.647113e+06  
1  [116, 80, 257, 21

In [20]:
initial_data = initial_data.sort_values(by='final_energy')

In [21]:
print(initial_data.iloc[0:5])

  set_iterations    time_taken completed_iterations  final_temp  \
1      900000000  57654.434814            900000000  159.267911   
0      900000000  55733.223443            900000000  159.267911   
4      900000000  62962.210565            900000000  159.267911   
2      900000000  56867.216628            900000000  159.267911   
5      900000000  59122.481962            900000000  159.267911   

                                       og_index_list     og_energy  \
1  [340, 411, 384, 90, 288, 44, 425, 2, 334, 178,...  1.202441e+08   
0  [109, 111, 463, 200, 56, 144, 456, 289, 307, 2...  8.630539e+07   
4  [9, 182, 469, 217, 173, 428, 265, 353, 113, 33...  1.501029e+08   
2  [73, 405, 145, 63, 463, 13, 300, 111, 377, 276...  1.080344e+08   
5  [216, 349, 282, 386, 438, 192, 324, 108, 335, ...  1.079082e+08   

                                          index_list  final_energy  
1  [116, 80, 257, 216, 93, 218, 396, 382, 206, 34...  1.643817e+06  
0  [116, 282, 452, 72, 67, 436, 447, 1

In [22]:
print(initial_data.iloc[0:10]['index_list'])

1    [116, 80, 257, 216, 93, 218, 396, 382, 206, 34...
0    [116, 282, 452, 72, 67, 436, 447, 199, 206, 41...
4    [234, 250, 17, 184, 227, 30, 428, 76, 107, 458...
2    [234, 205, 68, 427, 27, 236, 382, 424, 206, 40...
5    [234, 205, 235, 301, 227, 30, 473, 76, 107, 13...
6    [170, 205, 292, 315, 238, 112, 412, 70, 80, 13...
3    [164, 123, 400, 339, 167, 41, 195, 222, 344, 1...
Name: index_list, dtype: object


In [23]:
best_list = list()
for i in range(1):
    best_list.append(list(initial_data.iloc[0]['index_list']))
print(best_list)

[[116, 80, 257, 216, 93, 218, 396, 382, 206, 340, 20, 413, 19, 25, 352, 29, 48, 67, 447, 147, 249, 33, 452, 37, 337, 427, 168, 321, 324, 43, 469, 173, 119, 407, 155, 292, 388, 124, 87, 63, 88, 342, 109, 108, 360, 296, 157, 359, 30, 153, 55, 34, 97, 416, 314, 281, 239, 50, 448, 6, 345, 205, 195, 203, 170, 442, 243, 470, 115, 330, 104, 164, 169, 372, 199, 441, 54, 69, 111, 423, 260, 350, 307, 26, 362, 106, 75, 311, 24, 259, 180, 254, 268, 315, 335, 272, 298, 373, 64, 279, 367, 229, 118, 117, 411, 208, 344, 5, 451, 171, 263, 434, 219, 176, 35, 15, 472, 209, 393, 431, 172, 446, 94, 240, 450, 233, 83, 363, 143, 167, 297, 398, 127, 133, 454, 38, 303, 313, 193, 114, 343, 56, 148, 356, 325, 204, 188, 460, 187, 291, 338, 32, 2, 462, 49, 179, 312, 57, 162, 405, 432, 377, 228, 331, 334, 461, 456, 42, 65, 346, 387, 201, 9, 436, 419, 189, 192, 21, 395, 214, 28, 308, 58, 96, 211, 0, 444, 202, 68, 52, 91, 90, 181, 142, 369, 392, 381, 4, 191, 364, 294, 44, 322, 12, 105, 376, 59, 82, 354, 241, 394, 473

In [24]:
for j, index_list in enumerate(best_list):
    data = pd.DataFrame(columns=['residue number', 'amino acid', 'N', 'H', 'CA', 'CAPrime', 'CB', 'CBPrime', ])
    for i, pai in enumerate(index_list):
        residue_number = i + 1
        amino_acid = sequence_list[i]
    
        peak = peak_list[pai]
        assert pai == peak.get_data('peakNumber')
        n_shift = peak.get_data('TROSYNShift')
        h_shift = peak.get_data('TROSYHShift')
        ca_shift = peak.get_data('CAShift')
        ca_prime_shift = peak.get_data('CAPrimeShift')
        cb_shift = peak.get_data('CBShift')
        cb_prime_shift = peak.get_data('CBPrimeShift')
    
    
        temp_df = pd.DataFrame([[residue_number, amino_acid,
                                 n_shift, h_shift, ca_shift, ca_prime_shift, cb_shift, cb_prime_shift]],
                               
                               columns=['residue number', 'amino acid', 
                                        'N', 'H', 'CA', 'CAPrime', 'CB', 'CBPrime', ])
        data = data.append(temp_df, ignore_index=True)
    print(data.head())
    data.to_csv('/Volumes/Transcend/Fawzi_pycharm_project/data_{}.csv'.format('40:30'))

  residue number amino acid        N      H      CA CAPrime      CB CBPrime
0              1          T  122.329  8.622   60.33  49.504    None  63.778
1              2          P  124.950  8.669    None  59.305    None  57.487
2              3          K  118.145  7.480  58.599  65.716  57.822    None
3              4          D  122.497  7.587    None  58.724    None  59.062
4              5          D  122.893  8.752   54.95  56.258  53.153  54.189


3303    [234, 205, 257, 216, 322, 227, 457, 76, 107, 3...
Name: index_list, dtype: object